In [24]:
import pandas as pd
import MySQLdb
import numpy as np
import time
import datetime 
import random


d =14
time_a = pd.date_range('2016-09-01 00:00:00', periods=5760*(d+1), freq='15S')
# print time_a
time_b = pd.date_range('2016-09-01 00:00:14', periods=5760*(d+1), freq='15S')
# print time_b

clock1 = time_b.strftime("%H:%M:%S")

conn= MySQLdb.connect("localhost","conn","hems","collected_info")
c = conn.cursor()

conn1= MySQLdb.connect("localhost","conn","hems","device")
c1 = conn1.cursor()

####### set databases to empty #########
query_empty = """truncate table device.IRON""" 
c1.execute(query_empty)
print "device.IRON are Empty!!! \n"

query = "SELECT watt FROM collected_info.9_IRON WHERE no >= '%s'   AND no <= '%s'; "% (0,1539)
df = pd.read_sql(query, conn)
# print df.watt[5]


#### Random the turning-on time appliances
def rand_turnon():
    duration_sec =  random.randint(0, 59)
    duration_min =  random.randint(0, 59)
    duration_hour1 = ['10'] * 2 +  ['11'] * 3 + ['12'] * 5 + ['13'] * 50 + ['14'] * 20 + ['15'] * 10 +['16'] * 5 +['17'] * 5    #SET Start between 10-17 o'clock.
    duration_hour = random.choice(duration_hour1)     
    if 0 < duration_sec <= 15:
        k = 0
    elif 15 < duration_sec <= 30:
        k = 1
    elif 30 < duration_sec <= 45:
        k = 2
    else:
        k = 3
    off_item = (int(duration_hour)*240) + (int(duration_min)*4)+ k 
    print "time to turn-on appliacne on",duration_hour,":",duration_min,":",duration_sec,"equal to",off_item,"items"
    return off_item

 
#### Random the off-duration appliances 
def rand_duration():
    duration_sec =  random.randint(0, 59)
    duration_min =  random.randint(0, 59)
    duration_hour =  random.randint(2, 5)     
    seconds = (duration_hour*3600)+(duration_min*60)+duration_sec
    no_item = seconds/15
    print "consuming the appliance", duration_hour,"hours",duration_min,"minutes",duration_sec,"seconds that equal to",seconds,"seconds or",no_item,"items"
    return no_item


a = 0
days = 0
j = 0
i = 0
do_days1 = ['5'] * 33 + ['6'] * 33 + ['0'] * 34 
do_days = random.choice(do_days1)
dict = {'5': 'Friday', '6': 'Saturday', '0': 'Sunday'}
print "| use iron in next week on | ... ",dict[do_days]
off_item = rand_turnon()  
no_item = rand_duration()


while (j <= (5760*d)-1):
    no_days = int(time_b[j].strftime('%w'))  # find 0-6 of no_days

    
    if (no_days == 0 and clock1[j] == u'23:59:59'):    #if no_day is SUNDAY at 23:59:59 check d0_days in week 
        do_days1 = ['5'] * 33 + ['6'] * 33 + ['0'] * 34     #fri-sat-sun
        do_days = random.choice(do_days1) 
        dict = {'5': 'Friday', '6': 'Saturday', '0': 'Sunday'}
        print "\n| use iron in next week on | ... ",dict[do_days]
        off_item = rand_turnon()  
        no_item = rand_duration()


    if (int(do_days) == no_days):
        i= 0  
        c1.execute("INSERT INTO device.IRON (Timestamp, watt) VALUES (%s, %s)",(time_b[j],0))
        while (off_item+(5760*(days-1))<= j <= (no_item+off_item)+(5760*(days-1))):
            c1.execute("INSERT INTO device.IRON (Timestamp, watt) VALUES (%s, %s)",(time_b[j],df.watt[i]))
            conn1.commit()
            i += 1 
            j += 1     
            print "|",j,
    else : 
        c1.execute("INSERT INTO device.IRON (Timestamp, watt) VALUES (%s, %s)",(time_b[j],0))
        
    if (j%5760 == 0):
        print "\nDays :",days ,":",time_b[j].strftime('%a')
        days +=1        
    j += 1
    conn1.commit()
print "DONE!"

 


device.IRON are Empty!!! 

| use iron in next week on | ...  Sunday
time to turn-on appliacne on 10 : 27 : 55 equal to 2511 items
consuming the appliance 2 hours 49 minutes 37 seconds that equal to 10177 seconds or 678 items

Days : 0 : Thu

Days : 1 : Fri

Days : 2 : Sat

Days : 3 : Sun
| 19792 | 19793 | 19794 | 19795 | 19796 | 19797 | 19798 | 19799 | 19800 | 19801 | 19802 | 19803 | 19804 | 19805 | 19806 | 19807 | 19808 | 19809 | 19810 | 19811 | 19812 | 19813 | 19814 | 19815 | 19816 | 19817 | 19818 | 19819 | 19820 | 19821 | 19822 | 19823 | 19824 | 19825 | 19826 | 19827 | 19828 | 19829 | 19830 | 19831 | 19832 | 19833 | 19834 | 19835 | 19836 | 19837 | 19838 | 19839 | 19840 | 19841 | 19842 | 19843 | 19844 | 19845 | 19846 | 19847 | 19848 | 19849 | 19850 | 19851 | 19852 | 19853 | 19854 | 19855 | 19856 | 19857 | 19858 | 19859 | 19860 | 19861 | 19862 | 19863 | 19864 | 19865 | 19866 | 19867 | 19868 | 19869 | 19870 | 19871 | 19872 | 19873 | 19874 | 19875 | 19876 | 19877 | 19878 | 19879 | 19880

In [21]:
no_days = int(time_b[j].strftime('%w'))
print no_days

4


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  


# query = "SELECT watt FROM device.aircont WHERE Timestamp >= '%s'   AND Timestamp <= '%s'; "% (time_a[0],time_b[j-1])
query = "SELECT Timestamp,watt FROM device.IRON WHERE Timestamp >= '%s'   AND Timestamp <= '%s'; "% (time_a[0],time_b[j-1])
df = pd.read_sql(query, conn1, index_col=['Timestamp'])
fig, ax = plt.subplots()
ax.set_title('device.IRON')
df.plot(ax=ax )
plt.show()
df.to_csv('file.csv')
print len(df)


NameError: name 'time_a' is not defined